<a href="https://colab.research.google.com/github/Durso16/head_bet_/blob/main/bet_fix_sem_perda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas google-colab numpy

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd
from google.colab import files

# Passo 2: Carregar o arquivo de odds
file_path = '/content/odds_rodada24.csv'  # Substitua pelo caminho correto do arquivo de odds
odds_df = pd.read_csv(file_path)

# Função para calcular as apostas ideais e verificar lucro
def calcular_apostas_e_lucro(odds_a, odds_e, odds_b, max_investimento=20):
    investimento_a = max_investimento / (1 + (odds_b / odds_a) + (odds_e / odds_a))
    investimento_b = (odds_b / odds_a) * investimento_a
    investimento_e = (odds_e / odds_a) * investimento_a

    retorno_a = investimento_a * odds_a
    retorno_b = investimento_b * odds_b
    retorno_e = investimento_e * odds_e

    lucro_a = retorno_a - max_investimento
    lucro_b = retorno_b - max_investimento
    lucro_e = retorno_e - max_investimento

    # Lucro Total Combinado entre os três cenários
    lucro_total_combinado = lucro_a + lucro_b + lucro_e

    return investimento_a, investimento_e, investimento_b, lucro_a, lucro_e, lucro_b, lucro_total_combinado

# Função para verificar as condições e gerar resultados
def verificar_condicoes(odds_df, max_investimento, max_prejuizo_empate=20):
    resultados = []

    for i, row in odds_df.iterrows():
        time_a = row['TimeA']
        odds_a = row['Odds_TimeA']
        odds_e = row['Odds_Empate']
        time_b = row['TimeB']
        odds_b = row['Odds_TimeB']

        investimento_a, investimento_e, investimento_b, lucro_a, lucro_e, lucro_b, lucro_total_combinado = calcular_apostas_e_lucro(odds_a, odds_e, odds_b, max_investimento)

        # Marcar cenários com perda impossível (lucro total sempre positivo)
        perda_impossivel = lucro_total_combinado > 0

        resultado_jogo = {
            "Jogo": f"{time_a} vs {time_b}",
            "Odds_TimeA": odds_a,
            "Odds_Empate": odds_e,
            "Odds_TimeB": odds_b,
            "investimento_a": investimento_a,
            "investimento_b": investimento_b,
            "investimento_e": investimento_e,
            "lucro_a": lucro_a,
            "lucro_b": lucro_b,
            "lucro_e": lucro_e,
            "lucro_total_combinado": lucro_total_combinado,  # Lucro Total Combinado entre os três cenários
            "lucro_max_potencial": max(lucro_b, lucro_e),  # Maior ganho potencial (Vitória Time B ou Empate)
            "perda_impossivel": perda_impossivel  # Indicador de cenários sem perda
        }

        resultados.append(resultado_jogo)

    # Primeiro, ordenamos pelo Lucro Total Combinado de forma decrescente
    resultados.sort(key=lambda x: x['lucro_total_combinado'], reverse=True)

    # Em seguida, para jogos com Lucro Total Combinado igual, ordenamos pelo maior ganho potencial (Vitória Time B ou Empate)
    resultados.sort(key=lambda x: (x['lucro_total_combinado'], x['lucro_max_potencial']), reverse=True)

    return resultados

# Função para exibir e salvar resultados no formato desejado
def exibir_e_salvar_resultados(resultados):
    with open("resultados.txt", "w") as f:
        for resultado in resultados:
            status_perda = "Lucro Garantido (Sem Perda)" if resultado['perda_impossivel'] else "Risco de Perda"
            formato = (
                f"Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)\n"
                f"--------------------------------------------------------------------------------\n"
                f"Jogo: {resultado['Jogo']} \n"
                f"Odds: {resultado['Odds_TimeA']} (Time A), {resultado['Odds_Empate']} (Empate), {resultado['Odds_TimeB']} (Time B)\n\n"
                f"Vitória {resultado['Jogo'].split(' vs ')[0]:<15}R${resultado['investimento_a']:<20.2f}R${resultado['investimento_a'] + resultado['lucro_a']:<23.2f}R${resultado['lucro_a']:<.2f}\n"
                f"Vitória {resultado['Jogo'].split(' vs ')[1]:<15}R${resultado['investimento_b']:<20.2f}R${resultado['investimento_b'] + resultado['lucro_b']:<23.2f}R${resultado['lucro_b']:<.2f}\n"
                f"Empate              R${resultado['investimento_e']:<20.2f}R${resultado['investimento_e'] + resultado['lucro_e']:<23.2f}R${resultado['lucro_e']:<.2f}\n"
                f"\nLucro Total Combinado entre A, Empate, e B: R${resultado['lucro_total_combinado']:.2f}\n"
                f"Maior Ganho Potencial: R${resultado['lucro_max_potencial']:.2f}\n"
                f"Status: {status_perda}\n"
            )

            print(formato)
            f.write(formato + "\n")

    print("\nArquivo 'resultados.txt' gerado com sucesso!")

    # Download automático do arquivo na máquina local
    files.download("resultados.txt")

# Função principal para executar a análise
def executar_analise(tolerancia, max_investimento):
    resultados = verificar_condicoes(odds_df, max_investimento, max_prejuizo_empate=tolerancia)
    exibir_e_salvar_resultados(resultados)

# Defina a tolerância de prejuízo no empate e o valor total de investimento
tolerancia = 20  # Ajuste este valor conforme necessário
max_investimento = 20  # Ajuste este valor conforme necessário

# Executar a análise
executar_analise(tolerancia, max_investimento)


Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)
--------------------------------------------------------------------------------
Jogo: Cruzeiro vs Atlético Goianiense	 
Odds: 1.6 (Time A), 3.9 (Empate), 6.0 (Time B)

Vitória Cruzeiro       R$2.78                R$-12.77                 R$-15.55
Vitória Atlético Goianiense	R$10.43               R$53.04                  R$42.61
Empate              R$6.78                R$13.23                  R$6.45

Lucro Total Combinado entre A, Empate, e B: R$33.51
Maior Ganho Potencial: R$42.61
Status: Lucro Garantido (Sem Perda)

Cenário             Aposta no Time (R$)      Retorno Líquido (R$)     Lucro se o Time (R$)
--------------------------------------------------------------------------------
Jogo: Corinthians vs Flamengo 
Odds: 3.0 (Time A), 3.9 (Empate), 6.0 (Time B)

Vitória Corinthians    R$4.65                R$-1.40                  R$-6.05
Vitória Flamengo       R$9.30                R$45.12  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>